In [69]:
from sleap_roots import DicotPipeline, YoungerMonocotPipeline, OlderMonocotPipeline, Series, find_all_series
import pandas as pd
import numpy as np
from pathlib import Path

In [70]:
def generate_mermaid_graph_and_write_to_file(pipeline, name):
    trait_definitions = pipeline().define_traits()
    graph = "```mermaid\ngraph TD\n"
    unique_traits = set()

    for trait_def in trait_definitions:
        for input_trait in trait_def.input_traits:
            graph += f"    {input_trait} --> {trait_def.name}\n"
            unique_traits.add(input_trait)
            unique_traits.add(trait_def.name)

    graph += "```\n"
    
    # Write to a markdown file
    file_name = f"{name}_Mermaid_Graph.md"
    with open(file_name, 'w') as file:
        file.write(graph)
        file.write(f"\nDistinct Traits for {name}: {len(unique_traits)}\n")
    
    print(f"{name} Mermaid graph written to {file_name}")
    return graph, len(unique_traits)


In [71]:
canola_folder = Path("tests/data/canola_7do")
rice_10do_folder = Path("tests/data/rice_10do")
soy_folder = Path("tests/data/soy_6do")
rice_folder = Path("tests/data/rice_3do")

In [72]:
younger_monocot_pipeline = YoungerMonocotPipeline() # Requires crown and primary root predictions
older_monocot_pipeline = OlderMonocotPipeline() # Requires crown roots predictions
dicot_pipeline = DicotPipeline() # Requires primary and lateral root predictions

In [73]:
canola_data = find_all_series(canola_folder)
rice_10do_data = find_all_series(rice_10do_folder)
soy_data = find_all_series(soy_folder)
rice_data = find_all_series(rice_folder)

In [74]:
canola_data

['tests/data/canola_7do/919QDUH.h5']

In [75]:
print(f"number of series in canola: {len(canola_data)}")
print(f"number of series in rice_10do: {len(rice_10do_data)}")
print(f"number of series in soy: {len(soy_data)}")
print(f"number of series in rice: {len(rice_data)}")

number of series in canola: 1
number of series in rice_10do: 1
number of series in soy: 1
number of series in rice: 2


In [76]:
canola_series = [Series.load(s, primary_name="primary", lateral_name="lateral") for s in canola_data]
rice_10do_series = [Series.load(s, crown_name="crown") for s in rice_10do_data]
soy_series = [Series.load(s, primary_name="primary", lateral_name="lateral") for s in soy_data]
rice_series = [Series.load(s, primary_name="primary", crown_name="crown") for s in rice_data]

In [77]:
canola_df = dicot_pipeline.compute_batch_traits(canola_series, write_csv=True, csv_path="canola_batch_traits.csv")
rice_10do_df = older_monocot_pipeline.compute_batch_traits(rice_10do_series, write_csv=True, csv_path="rice_10do_batch_traits.csv")
soy_df = dicot_pipeline.compute_batch_traits(soy_series, write_csv=True, csv_path="soy_batch_traits.csv")
rice_df = younger_monocot_pipeline.compute_batch_traits(rice_series, write_csv=True, csv_path="rice_batch_traits.csv")

In [78]:
print("Shape of canola dataframe: ", canola_df.shape)
print("Shape of rice_10do dataframe: ", rice_10do_df.shape)
print("Shape of soy dataframe: ", soy_df.shape)
print("Shape of rice dataframe: ", rice_df.shape)

Shape of canola dataframe:  (1, 1036)
Shape of rice_10do dataframe:  (1, 901)
Shape of soy dataframe:  (1, 1036)
Shape of rice dataframe:  (2, 919)


In [79]:
pipelines = {
    "DicotPipeline": DicotPipeline,
    "YoungerMonocotPipeline": YoungerMonocotPipeline,
    "OlderMonocotPipeline": OlderMonocotPipeline,
}

for name, pipeline in pipelines.items():
    graph, distinct_traits = generate_mermaid_graph_and_write_to_file(pipeline, name)
    print(f"Mermaid Graph for {name}:")
    print(graph)
    print(f"Distinct Traits for {name}: {distinct_traits}\n")

DicotPipeline Mermaid graph written to DicotPipeline_Mermaid_Graph.md
Mermaid Graph for DicotPipeline:
```mermaid
graph TD
    primary_pts --> primary_max_length_pts
    primary_max_length_pts --> pts_all_array
    lateral_pts --> pts_all_array
    primary_max_length_pts --> pts_list
    lateral_pts --> pts_list
    primary_max_length_pts --> root_widths
    lateral_pts --> root_widths
    lateral_pts --> lateral_count
    lateral_pts --> lateral_proximal_node_inds
    lateral_pts --> lateral_distal_node_inds
    lateral_pts --> lateral_lengths
    lateral_pts --> lateral_base_pts
    lateral_pts --> lateral_tip_pts
    pts_list --> scanline_intersection_counts
    lateral_pts --> lateral_angles_distal
    lateral_distal_node_inds --> lateral_angles_distal
    lateral_pts --> lateral_angles_proximal
    lateral_proximal_node_inds --> lateral_angles_proximal
    network_length --> network_solidity
    chull_area --> network_solidity
    pts_all_array --> ellipse
    pts_all_array --> bo